# Pandas Avanzado - Ejemplos Prácticos

Este notebook contiene ejemplos ejecutables de los conceptos avanzados de pandas cubiertos en la teoría.

![![image.png](attachment:image.png)](attachment:image.png)

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas exitosamente")

## 📊 Ejemplo 1: Series Temporales y Análisis de Fechas

![📊 Ejemplo 1: Series Temporales y Análisis de Fechas](imgs/series_temporales.png)


In [ ]:
# Crear datos de ejemplo para análisis temporal
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
np.random.seed(42)

# Datos de ventas con tendencia y estacionalidad
ventas_data = {
    'fecha': dates,
    'ventas': 100 + np.cumsum(np.random.normal(0, 10, len(dates))) + 20 * np.sin(2 * np.pi * np.arange(len(dates)) / 365.25),
    'categoria': np.random.choice(['A', 'B', 'C'], len(dates)),
    'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], len(dates))
}

df_temporal = pd.DataFrame(ventas_data)
df_temporal['fecha'] = pd.to_datetime(df_temporal['fecha'])
df_temporal.set_index('fecha', inplace=True)

print("Dataset temporal creado:")
print(df_temporal.head())
print(f"\nForma del dataset: {df_temporal.shape}")

In [ ]:
# Resampling - Agrupar por mes
monthly_sales = df_temporal['ventas'].resample('M').sum()
print("Ventas mensuales:")
print(monthly_sales.head())

In [ ]:
# Rolling windows - Promedios móviles
df_temporal['ventas_ma7'] = df_temporal['ventas'].rolling(window=7).mean()
df_temporal['ventas_ma30'] = df_temporal['ventas'].rolling(window=30).mean()

print("Datos con promedios móviles:")
print(df_temporal[['ventas', 'ventas_ma7', 'ventas_ma30']].head(10))

In [ ]:
# Análisis de diferencias y cambios porcentuales
df_temporal['ventas_diff'] = df_temporal['ventas'].diff()
df_temporal['ventas_pct_change'] = df_temporal['ventas'].pct_change()

print("Análisis de cambios:")
print(df_temporal[['ventas', 'ventas_diff', 'ventas_pct_change']].head())

## 🔗 Ejemplo 2: Análisis de Correlaciones

![🔗 Ejemplo 2: Análisis de Correlaciones](imgs/correlaciones.png)


In [ ]:
# Crear dataset con variables correlacionadas
np.random.seed(123)
n_samples = 1000

data_corr = {
    'ventas': np.random.normal(1000, 200, n_samples),
    'marketing': np.random.normal(500, 100, n_samples),
    'temperatura': np.random.normal(20, 5, n_samples),
    'ventas_altas': 100 + 0.8 * np.random.normal(1000, 200, n_samples) + np.random.normal(0, 50, n_samples)
}

df_corr = pd.DataFrame(data_corr)

print("Dataset para análisis de correlaciones:")
print(df_corr.head())
print(f"\nEstadísticas descriptivas:")
print(df_corr.describe())

In [ ]:
# Matriz de correlación
correlation_matrix = df_corr.corr()
print("Matriz de correlación:")
print(correlation_matrix.round(3))

In [ ]:
# Identificar variables altamente correlacionadas
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:  # Umbral de correlación alta
            high_corr_pairs.append((
                correlation_matrix.columns[i], 
                correlation_matrix.columns[j], 
                corr_value
            ))

print("Pares de variables altamente correlacionadas (|r| > 0.5):")
for var1, var2, corr in high_corr_pairs:
    print(f"{var1} - {var2}: {corr:.3f}")

## 🔍 Ejemplo 3: Valores Faltantes y Imputación

![🔍 Ejemplo 3: Valores Faltantes y Imputación](imgs/valores_faltantes.png)


In [ ]:
# Crear dataset con valores faltantes
np.random.seed(456)
df_missing = pd.DataFrame({
    'categoria': np.random.choice(['A', 'B', 'C'], 1000),
    'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], 1000),
    'ventas': np.random.normal(1000, 200, 1000),
    'precio': np.random.normal(50, 10, 1000),
    'descuento': np.random.normal(0.1, 0.05, 1000)
})

# Introducir valores faltantes de forma selectiva
df_missing.loc[np.random.choice(df_missing.index, 100, replace=False), 'ventas'] = np.nan
df_missing.loc[np.random.choice(df_missing.index, 80, replace=False), 'precio'] = np.nan
df_missing.loc[np.random.choice(df_missing.index, 60, replace=False), 'descuento'] = np.nan

print("Dataset con valores faltantes:")
print(df_missing.info())
print("\nPorcentaje de valores faltantes por columna:")
missing_percent = (df_missing.isnull().sum() / len(df_missing)) * 100
print(missing_percent.round(2))

In [ ]:
# Imputación por grupos
print("Ejemplo de imputación por categoría:")
ventas_by_category = df_missing.groupby('categoria')['ventas'].mean()
print("Ventas promedio por categoría:")
print(ventas_by_category)

# Aplicar imputación
df_imputed = df_missing.copy()
df_imputed['ventas'] = df_imputed['ventas'].fillna(
    df_imputed.groupby('categoria')['ventas'].transform('mean')
)

In [ ]:
# Interpolación temporal (para datos con índice temporal)
# Crear datos con fechas
dates_missing = pd.date_range(start='2023-01-01', periods=100, freq='D')
df_temporal_missing = pd.DataFrame({
    'fecha': dates_missing,
    'ventas': np.random.normal(100, 20, 100)
})

# Introducir valores faltantes
df_temporal_missing.loc[np.random.choice(df_temporal_missing.index, 20, replace=False), 'ventas'] = np.nan
df_temporal_missing.set_index('fecha', inplace=True)

print("Datos antes de interpolación:")
print(df_temporal_missing.head(10))

# Aplicar interpolación temporal
df_temporal_missing['ventas_interpolated'] = df_temporal_missing['ventas'].interpolate(method='time')

print("\nDatos después de interpolación:")
print(df_temporal_missing.head(10))

## 📊 Ejemplo 4: Pivot Tables Complejas

![📊 Ejemplo 4: Pivot Tables Complejas](imgs/pivot_tables.png)


In [ ]:
# Crear dataset para pivot tables
np.random.seed(789)
df_pivot = pd.DataFrame({
    'fecha': pd.date_range('2023-01-01', periods=365, freq='D'),
    'categoria': np.random.choice(['Electrónicos', 'Ropa', 'Hogar'], 365),
    'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], 365),
    'ventas': np.random.normal(1000, 300, 365),
    'unidades': np.random.normal(100, 30, 365)
})

df_pivot['fecha'] = pd.to_datetime(df_pivot['fecha'])
df_pivot['mes'] = df_pivot['fecha'].dt.month

print("Dataset para pivot tables:")
print(df_pivot.head())
print(f"\nForma del dataset: {df_pivot.shape}")

In [ ]:
# Pivot table básica
pivot_basic = df_pivot.pivot_table(
    index='categoria',
    columns='region',
    values='ventas',
    aggfunc='sum'
)

print("Pivot table básica - Ventas por Categoría y Región:")
print(pivot_basic.round(0))

In [ ]:
# Pivot table con múltiples agregaciones
pivot_multi = df_pivot.pivot_table(
    index='categoria',
    columns='region',
    values=['ventas', 'unidades'],
    aggfunc={'ventas': ['sum', 'mean'], 'unidades': ['sum', 'mean']},
    fill_value=0
)

print("Pivot table con múltiples agregaciones:")
print(pivot_multi.round(2))

In [ ]:
# Pivot table con totales
pivot_with_totals = df_pivot.pivot_table(
    index='categoria',
    columns='region',
    values='ventas',
    aggfunc='sum',
    margins=True,
    margins_name='Total'
)

print("Pivot table con totales:")
print(pivot_with_totals.round(0))

## 🔤 Ejemplo 5: Manejo de Texto y Datos Categóricos

![🔤 Ejemplo 5: Manejo de Texto y Datos Categóricos](imgs/visualizacion.png)


In [ ]:
# Crear dataset con datos de texto
df_text = pd.DataFrame({
    'email': [
        'usuario1@gmail.com',
        'admin@empresa.com',
        'test@dominio.es',
        'usuario2@yahoo.com'
    ],
    'telefono': [
        '+34 123 456 789',
        '123-456-7890',
        '(555) 123-4567',
        '555.123.4567'
    ],
    'descripcion': [
        'Producto de ALTA CALIDAD',
        'Artículo en OFERTA especial',
        'Nuevo producto disponible',
        'Artículo con descuento del 20%'
    ],
    'categoria': ['A', 'B', 'A', 'C']
})

print("Dataset con datos de texto:")
print(df_text)

In [ ]:
# Operaciones básicas de texto
df_text['email_lower'] = df_text['email'].str.lower()
df_text['descripcion_clean'] = df_text['descripcion'].str.lower()
df_text['descripcion_length'] = df_text['descripcion'].str.len()

print("Operaciones básicas de texto:")
print(df_text[['email', 'email_lower', 'descripcion_length']])

In [ ]:
# Extracción con expresiones regulares
df_text['dominio'] = df_text['email'].str.extract(r'@([^.]+)')
df_text['proveedor'] = df_text['email'].str.extract(r'@([^.]+)\.')

# Extraer números de teléfono limpios
df_text['telefono_limpio'] = df_text['telefono'].str.replace(r'[^\d]', '', regex=True)

print("Extracción con expresiones regulares:")
print(df_text[['email', 'dominio', 'proveedor', 'telefono', 'telefono_limpio']])

## ⚡ Ejemplo 6: Performance y Optimización

![⚡ Ejemplo 6: Performance y Optimización](imgs/performance.png)


In [ ]:
# Crear dataset grande para testing
np.random.seed(101)
n_rows = 100000

df_large = pd.DataFrame({
    'col1': np.random.randint(1, 100, n_rows),
    'col2': np.random.randint(1, 100, n_rows),
    'col3': np.random.randn(n_rows),
    'categoria': np.random.choice(['A', 'B', 'C', 'D'], n_rows)
})

print(f"Dataset creado con {n_rows:,} filas")
print(f"Uso de memoria: {df_large.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Demostrar operaciones vectorizadas vs loops
import time

print("Comparación de performance:")

# Método 1: Loop (más lento)
start_time = time.time()
result_loop = []
for i in range(len(df_large)):
    result_loop.append(df_large.iloc[i]['col1'] * df_large.iloc[i]['col2'])
loop_time = time.time() - start_time

# Método 2: Vectorizado (más rápido)
start_time = time.time()
result_vectorized = df_large['col1'] * df_large['col2']
vectorized_time = time.time() - start_time

print(f"Loop: {loop_time:.4f} segundos")
print(f"Vectorizado: {vectorized_time:.4f} segundos")
print(f"Mejora: {loop_time/vectorized_time:.1f}x más rápido")

In [ ]:
# Optimización de tipos de datos
print("Optimización de tipos de datos:")

# Ver uso actual de memoria
memory_before = df_large.memory_usage(deep=True).sum() / 1024**2
print(f"Memoria antes: {memory_before:.2f} MB")

# Optimizar tipos de datos
df_optimized = df_large.copy()
df_optimized['col1'] = df_optimized['col1'].astype('int16')
df_optimized['col2'] = df_optimized['col2'].astype('int16')
df_optimized['categoria'] = df_optimized['categoria'].astype('category')

memory_after = df_optimized.memory_usage(deep=True).sum() / 1024**2
print(f"Memoria después: {memory_after:.2f} MB")
print(f"Ahorro: {((memory_before - memory_after) / memory_before) * 100:.1f}%")

## 🔗 Ejemplo 7: Joins y Merges Avanzados

![🔗 Ejemplo 7: Joins y Merges Avanzados](imgs/joins.png)


In [ ]:
# Crear datasets para merge
df_customers = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Ana', 'Luis', 'María', 'Carlos', 'Elena'],
    'city': ['Madrid', 'Barcelona', 'Valencia', 'Sevilla', 'Bilbao']
})

df_orders = pd.DataFrame({
    'order_id': [101, 102, 103, 104, 105, 106],
    'customer_id': [1, 2, 1, 3, 6, 2],  # customer_id=6 no existe en customers
    'amount': [150.50, 200.00, 75.25, 300.00, 50.00, 120.75],
    'order_date': ['2023-01-15', '2023-01-16', '2023-01-17', 
                   '2023-01-18', '2023-01-19', '2023-01-20']
})

print("Dataset de clientes:")
print(df_customers)
print("\nDataset de órdenes:")
print(df_orders)

In [ ]:
# Diferentes tipos de merge
print("INNER JOIN:")
inner_merge = pd.merge(df_customers, df_orders, on='customer_id', how='inner')
print(inner_merge)

print("\nLEFT JOIN:")
left_merge = pd.merge(df_customers, df_orders, on='customer_id', how='left')
print(left_merge)

print("\nRIGHT JOIN:")
right_merge = pd.merge(df_customers, df_orders, on='customer_id', how='right')
print(right_merge)

print("\nOUTER JOIN:")
outer_merge = pd.merge(df_customers, df_orders, on='customer_id', how='outer')
print(outer_merge)

## 🎯 Ejemplo 8: Detección de Outliers

![🎯 Ejemplo 8: Detección de Outliers](imgs/outliers.png)


In [ ]:
# Crear dataset con outliers
np.random.seed(202)
normal_data = np.random.normal(100, 15, 1000)
outliers = np.array([200, 250, -50, 300, 180])  # Valores extremos
data_with_outliers = np.concatenate([normal_data, outliers])

df_outliers = pd.DataFrame({
    'valor': data_with_outliers,
    'categoria': np.random.choice(['A', 'B', 'C'], len(data_with_outliers))
})

print(f"Dataset con {len(data_with_outliers)} valores (incluye outliers)")
print("Estadísticas descriptivas:")
print(df_outliers['valor'].describe())

In [ ]:
# Detección de outliers usando IQR
Q1 = df_outliers['valor'].quantile(0.25)
Q3 = df_outliers['valor'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"Límite inferior: {lower_bound:.2f}")
print(f"Límite superior: {upper_bound:.2f}")

# Identificar outliers
outliers_iqr = df_outliers[(df_outliers['valor'] < lower_bound) | 
                          (df_outliers['valor'] > upper_bound)]

print(f"\nNúmero de outliers detectados: {len(outliers_iqr)}")
print("Valores outliers:")
print(outliers_iqr['valor'].values)

In [ ]:
# Tratamiento de outliers - winsorizing
df_cleaned = df_outliers.copy()
df_cleaned['valor_winsorized'] = df_cleaned['valor'].clip(
    lower=lower_bound, 
    upper=upper_bound
)

print("Comparación antes y después del winsorizing:")
comparison = pd.DataFrame({
    'Original': df_outliers['valor'].describe(),
    'Winsorized': df_cleaned['valor_winsorized'].describe()
})
print(comparison.round(2))

## 📊 Ejemplo 9: Visualización con Pandas

![📊 Ejemplo 9: Visualización con Pandas](imgs/visualizacion.png)


In [ ]:
# Crear dataset para visualización
np.random.seed(303)
dates_viz = pd.date_range('2023-01-01', periods=100, freq='D')
df_viz = pd.DataFrame({
    'fecha': dates_viz,
    'ventas': 100 + np.cumsum(np.random.normal(0, 5, 100)),
    'marketing': 50 + np.random.normal(0, 10, 100),
    'categoria': np.random.choice(['A', 'B', 'C'], 100)
})
df_viz.set_index('fecha', inplace=True)

print("Dataset para visualización:")
print(df_viz.head())

# Crear subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análisis Visual con Pandas', fontsize=16)

# Gráfico de línea
df_viz['ventas'].plot(ax=axes[0,0], title='Ventas a lo largo del tiempo')
axes[0,0].set_ylabel('Ventas')

# Histograma
df_viz['ventas'].plot.hist(ax=axes[0,1], bins=20, title='Distribución de Ventas')
axes[0,1].set_xlabel('Ventas')

# Box plot por categoría
df_viz.boxplot(column='ventas', by='categoria', ax=axes[1,0])
axes[1,0].set_title('Ventas por Categoría')

# Scatter plot
df_viz.plot.scatter(x='marketing', y='ventas', ax=axes[1,1], title='Marketing vs Ventas')

plt.tight_layout()
plt.show()

## 🔢 Ejemplo 10: MultiIndex y Estructuras Complejas

![🔢 Ejemplo 10: MultiIndex y Estructuras Complejas](imgs/multiindex.png)


In [ ]:
# Crear MultiIndex
arrays = [
    ['A', 'A', 'B', 'B', 'C', 'C'],
    ['uno', 'dos', 'uno', 'dos', 'uno', 'dos']
]
index = pd.MultiIndex.from_arrays(arrays, names=['letra', 'numero'])

df_multi = pd.DataFrame(
    np.random.randn(6, 3),
    index=index,
    columns=['col1', 'col2', 'col3']
)

print("DataFrame con MultiIndex:")
print(df_multi)

In [ ]:
# Acceso a MultiIndex
print("Acceso por primer nivel ('A'):")
print(df_multi.loc['A'])

print("\nAcceso por ambos niveles ('A', 'uno'):")
print(df_multi.loc[('A', 'uno')])

print("\nCross-section por 'numero' = 'dos':")
print(df_multi.xs('dos', level='numero'))

In [ ]:
# Crear pivot table con MultiIndex
np.random.seed(404)
df_multi_pivot = pd.DataFrame({
    'year': [2022, 2022, 2023, 2023] * 3,
    'quarter': [1, 2, 1, 2] * 3,
    'region': ['Norte', 'Sur'] * 6,
    'sales': np.random.randint(100, 1000, 12)
})

pivot_multi = df_multi_pivot.pivot_table(
    index=['year', 'quarter'],
    columns='region',
    values='sales',
    aggfunc='sum'
)

print("Pivot table con MultiIndex en filas:")
print(pivot_multi)

## 📝 Resumen de Conceptos Cubiertos

![📝 Resumen de Conceptos Cubiertos](imgs/visualizacion.png)


En este notebook hemos demostrado:

✅ **Series Temporales**: Resampling, rolling windows, análisis de diferencias
✅ **Correlaciones**: Matrices de correlación, identificación de relaciones fuertes
✅ **Valores Faltantes**: Imputación por grupos, interpolación temporal
✅ **Pivot Tables**: Agregaciones múltiples, totales, funciones personalizadas
✅ **Procesamiento de Texto**: Expresiones regulares, limpieza de datos
✅ **Performance**: Operaciones vectorizadas, optimización de memoria
✅ **Joins**: Inner, left, right, outer joins con diferentes estrategias
✅ **Outliers**: Detección IQR, tratamiento con winsorizing
✅ **Visualización**: Múltiples tipos de gráficos, subplots
✅ **MultiIndex**: Creación, acceso, pivot tables complejas

Estos conceptos son fundamentales para proyectos profesionales de data science y machine learning.